# Phenotype data imputation

FIXME: need to rework it based on Zining's latest work and also taking whatever useful from `phenotype_formatting.ipynb`

In [ ]:
 nohup sos run /mnt/mfs/hgrcgrid/homes/zq2209/missForest.ipynb miss_forest \
    --phenoFile /mnt/mfs/hgrcgrid/homes/zq2209/proteomics/rosmap/test1.bed.gz \
    --cwd /mnt/mfs/hgrcgrid/homes/zq2209/proteomics/rosmap/ \
    --mem 40G \
    --walltime 100h

In [2]:
[global]
import os
# Work directory & output directory
parameter: cwd = path("output")
# The filename namefor output data
parameter: container = ''
parameter: entrypoint={('micromamba run -n' + ' ' + container.split('/')[-1][:-4]) if container.endswith('.sif') else f''}
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "72h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20
# Path to the input molecular phenotype data.
parameter: phenoFile = paths
# name for the analysis output
parameter: name= f'{phenoFile:bn}'
# Whether the input data is named by gene_id or gene_name. By default it is gene_id, if not, please change it to gene_name
parameter: phenotype_id_type = 'gene_id'
gene_name_as_phenotype_id = "gene_name" == phenotype_id_type

In [1]:
[miss_forest]
input: phenoFile
output: f'{_input:nn}.filter_na.bed.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container, entrypoint=entrypoint
   library("tibble")
   library("readr")
   library("dplyr")
   library(missForest)
  
    bed = read_delim("${_input}")
    mtx = as.matrix(bed[,5:ncol(bed)])
    rownames(mtx) = unlist(bed[,4])
    tbl_filtered = missForest::missForest(mtx)
    tbl_filtered = as_tibble(tbl_filtered$ximp, rownames = colnames(bed)[4])  
    bed_filtered = inner_join(bed[,1:4],tbl_filtered)
    write_delim(bed_filtered, "${_output:n}", "\t" )

bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container, entrypoint=entrypoint
    bgzip -f ${_output:n}
    tabix ${_output}

bash: expand= "$[ ]", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container = container, entrypoint=entrypoint
        stdout=$[_output:n].stdout
        for i in $[_output] ; do 
        echo "output_info: $i " >> $stdout;
        echo "output_size:" `ls -lh $i | cut -f 5  -d  " "`   >> $stdout;
        echo "output_rows:" `cat $i | wc -l  | cut -f 1 -d " "`   >> $stdout;
        echo "output_column:" `cat $i | head -1 | wc -w `   >> $stdout;
        echo "output_headerow:" `cat $i | grep "##" | wc -l `   >> $stdout;
        echo "output_preview:"   >> $stdout;
        cat $i  | grep -v "##" | head  | cut -f 1,2,3,4,5,6   >> $stdout ; done

In [ ]:
[xgboost]
input: phenoFile
output: f'{_input:nn}.filter_na.bed.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container, entrypoint=entrypoint
   library("tibble")
   library("readr")
   library("dplyr")
   library(mixgb)
  
    bed = read_delim("${_input}")
    mtx = bed[,5:ncol(bed)]
    rownames(mtx) = unlist(bed[,4])
    tbl_filtered = mixgb::mixgb(mtx)
    tbl_filtered = as_tibble(tbl_filtered$ximp, rownames = colnames(bed)[4])  
    bed_filtered = inner_join(bed[,1:4],tbl_filtered)
    write_delim(bed_filtered, "${_output:n}", "\t" )

bash: expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container=container, entrypoint=entrypoint
    bgzip -f ${_output:n}
    tabix ${_output}
bash: expand= "$[ ]", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', container = container, entrypoint=entrypoint
        stdout=$[_output:n].stdout
        for i in $[_output] ; do 
        echo "output_info: $i " >> $stdout;
        echo "output_size:" `ls -lh $i | cut -f 5  -d  " "`   >> $stdout;
        echo "output_rows:" `cat $i | wc -l  | cut -f 1 -d " "`   >> $stdout;
        echo "output_column:" `cat $i | head -1 | wc -w `   >> $stdout;
        echo "output_headerow:" `cat $i | grep "##" | wc -l `   >> $stdout;
        echo "output_preview:"   >> $stdout;
        cat $i  | grep -v "##" | head  | cut -f 1,2,3,4,5,6   >> $stdout ; done